In [1]:
%load_ext autoreload
%autoreload 2
from module_test.raw_code.optionlib_2.vol.implied_vol import (
    estimate_crr_implied_volatility,
    vol_est_brute_force_bjs_2002,
    vector_vol_estimation
)
from module_test.raw_code.optionlib_2.assets.forward import (
    EquityForward, 
    time_distance_helper,
    get_vectorized_dividend_rate,
    get_vectorized_dividend_scehdule
)

from module_test.raw_code.optionlib_2.assets.dividend import (
    vector_convert_to_time_frac
)

from module_test.raw_code.optionlib_2.greeks.numerical.bjs2002 import (
    bjs2002_numerical_greeks,
)

from module_test.raw_code.optionlib_2.greeks.numerical.binomial import (
    binomial_tree_greeks,
)
from datetime import datetime
from dbase.DataAPI.ThetaData import retrieve_chain_bulk
from trade.helpers.helper import change_to_last_busday, retrieve_timeseries
import os
os.environ['PROXY_URL'] = ''
def get_spot(tick, date):
    return retrieve_timeseries(tick, date, date)['close'][0]
test_start, test_valuation_date = '2025-07-16', '2025-07-16'


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-07-24 23:43:20 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://54.144.4.219:5500/thetadata


In [2]:
import pandas as pd
pd.options.plotting.backend = "plotly"

In [44]:
aapl_chain=retrieve_chain_bulk(
    'AAPL',
    0,
    change_to_last_busday(test_valuation_date),
    change_to_last_busday(test_valuation_date),
    '16:00'

)
rates = 0.0423199987411499
S = get_spot('AAPL', (test_valuation_date))
valuation_dates = [test_valuation_date] * len(aapl_chain)
end_dates = aapl_chain['Expiration'].tolist()
r = [rates] * len(aapl_chain)
s = [S] * len(aapl_chain)
tickers = ['AAPL'] * len(aapl_chain)
T = [time_distance_helper(end_dates[i], valuation_dates[i]) for i in range(len(aapl_chain))]


q = get_vectorized_dividend_rate(
    tickers=tickers,
    spots=s,
    valuation_dates=valuation_dates,
)


discrete_q = get_vectorized_dividend_scehdule(
    tickers=['AAPL'] * len(aapl_chain),
    valuation_dates=[test_valuation_date] * len(aapl_chain),
    end_dates=aapl_chain['Expiration'].tolist(),
    start_dates=[test_valuation_date] * len(aapl_chain),
)

discrete_q_convert = vector_convert_to_time_frac(
    discrete_q, 
    valuation_dates=[test_valuation_date] * len(aapl_chain), 
    end_dates=aapl_chain['Expiration'].tolist(), 
)

In [50]:
import numpy as np
np.array(discrete_q_convert)[:4]

array([<Schedule: 0 dividends>, <Schedule: 0 dividends>,
       <Schedule: 0 dividends>, <Schedule: 0 dividends>], dtype=object)

In [4]:
vector_params = list(zip(
        s, aapl_chain['Strike'].tolist(),
    T, r, aapl_chain['Midpoint'], 
    q, aapl_chain['Right'].str.lower().tolist(),))

In [5]:
vol_batch_bjs = vector_vol_estimation(
    vol_est_brute_force_bjs_2002,
    vector_params
)
vol_batch_bjs

[0.13222703067576688,
 0.27657676441911044,
 0.2802011300282507,
 0.14197532438310956,
 0.17022038050951271,
 0.2707027925698142,
 0.2655786894672367,
 0.22921005525138127,
 0.2627041926048151,
 0.26782829570739264,
 0.19671574289357233,
 0.24333258331458285,
 0.2763268081702042,
 0.058489937248431205,
 0.39018187954698863,
 0.28182584564614116,
 0.026995449886247157,
 0.3173196329908248,
 0.24133293332333305,
 0.02062156553913848,
 0.268203230080752,
 0.2725774644366109,
 0.2550805270131753,
 0.271577639440986,
 0.2769516987924698,
 0.40392947323683087,
 0.40767881697042424,
 0.27570191754793866,
 0.2439574739368484,
 0.2752020050501262,
 0.4956634165854146,
 0.03661876546913673,
 0.3116956173904347,
 0.07848643716092901,
 0.2971981549538738,
 0.0153724843121078,
 0.0153724843121078,
 0.3145701142528563,
 0.27207755193879846,
 0.30757133928348207,
 0.28057606440161004,
 0.2747020925523138,
 0.018496937423435587,
 0.01637230930773269,
 0.015497462436560912,
 0.0153724843121078,
 0.2715

In [31]:
from module_test.raw_code.optionlib_2.utils.batch_operation import vector_batch_processor
slc = slice(None)

test2 = vector_batch_processor(
  vector_vol_estimation,
    vol_est_brute_force_bjs_2002,
    # vector_params,
    None,
    s[slc], 
    aapl_chain['Strike'].tolist()[slc],
    T[slc], 
    r[slc], 
    aapl_chain['Midpoint'].tolist()[slc], 
    q[slc], 
    aapl_chain['Right'].str.lower().tolist()[slc],
      
)

In [35]:
aapl_chain['bjs_vol_non_batch'] = vol_batch_bjs
aapl_chain['bjs_vol_batch'] = test2
aapl_chain['diff'] = aapl_chain['bjs_vol_non_batch'] - aapl_chain['bjs_vol_batch']
aapl_chain['bjs_vol_non_batch'].equals(aapl_chain['bjs_vol_batch'])

True

In [36]:
aapl_chain['bjs_vol'] = vol_batch_bjs
aapl_chain

,Root,Expiration,Strike,Right,Bid_size,CloseBid,Ask_size,CloseAsk,Date,Midpoint,Weighted_midpoint,bjs_vol_non_batch,bjs_vol_batch,diff,bjs_vol
datetime,,,,,,,,,,,,,,,
2025-07-16,AAPL,2025-08-22,215.0,P,1,9.75,38,10.15,20250716,9.950,10.139744,0.132227,0.132227,0.0,0.132227
2025-07-16,AAPL,2025-08-29,215.0,C,2,6.25,1,6.40,20250716,6.325,6.300000,0.276577,0.276577,0.0,0.276577
2025-07-16,AAPL,2025-08-22,215.0,C,2,5.65,1,5.75,20250716,5.700,5.683333,0.280201,0.280201,0.0,0.280201
2025-07-16,AAPL,2025-08-29,215.0,P,24,9.90,23,10.60,20250716,10.250,10.242553,0.141975,0.141975,0.0,0.141975
2025-07-16,AAPL,2025-09-19,215.0,P,8,11.25,1,11.40,20250716,11.325,11.266667,0.170220,0.170220,0.0,0.170220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-16,AAPL,2025-07-25,215.0,P,4,5.90,30,6.10,20250716,6.000,6.076471,0.017747,0.017747,0.0,0.017747
2025-07-16,AAPL,2025-08-08,215.0,C,6,4.45,12,4.55,20250716,4.500,4.516667,0.302947,0.302947,0.0,0.302947
2025-07-16,AAPL,2025-08-08,215.0,P,4,8.55,6,8.85,20250716,8.700,8.730000,0.045867,0.045867,0.0,0.045867


In [40]:
estimate_crr_implied_volatility?
crr_vector_params_discrete = list(zip(
    s, aapl_chain['Strike'].tolist(), ## Spot, Strike
    T, r,  ## Time to Maturity, Risk Free Rate
    aapl_chain['Midpoint'], ## Midpoint Price
    discrete_q_convert, ## Discrete Dividend Schedules
    aapl_chain['Right'].str.lower().tolist(), ## Option Type
    [100] * len(aapl_chain),  ## Number of Steps
    ['discrete'] * len(aapl_chain), ## Dividend Type
    [True] * len(aapl_chain),)) ## American==True, European==False

crr_vector_params_cont = list(zip(
    s, aapl_chain['Strike'].tolist(), ## Spot, Strike
    T, r,  ## Time to Maturity, Risk Free Rate
    aapl_chain['Midpoint'], ## Midpoint Price
    q, ## Discrete Dividend Schedules
    aapl_chain['Right'].str.lower().tolist(), ## Option Type
    [100] * len(aapl_chain),  ## Number of Steps
    ['continuous'] * len(aapl_chain), ## Dividend Type
    [True] * len(aapl_chain),)) ## American==True, European==False

Signature:
estimate_crr_implied_volatility(
    S: float,
    K: float,
    T: float,
    r: float,
    market_price: float,
    q: float = 0.0,
    option_type: str = 'c',
    N: int = 1000,
    dividend_type: Literal['continuous', 'discrete'] = 'continuous',
    american: bool = False,
) -> float
Docstring:
Estimate implied volatility using optimization.

Parameters:
- S: Spot price
- K: Strike price
- T: Time to maturity
- r: Risk-free interest rate
- market_price: Market price of the option
- q: Continuous dividend yield (default is 0.0)
- option_type: 'c' for call, 'p' for put
- N: Number of time steps in the binomial tree

Returns:
- Estimated volatility
File:      ~/cloned_repos/QuantTools/module_test/raw_code/optionlib_2/vol/implied_vol.py
Type:      function

In [41]:
import time
start = time.time()
vol_batch_crr_discrete = vector_batch_processor(
    vector_vol_estimation,
    estimate_crr_implied_volatility,
    crr_vector_params_discrete,
)

print(f"Finished Discrete in {time.time() - start} seconds")
start = time.time()
vol_batch_crr_cont = vector_batch_processor(
    vector_vol_estimation,
    estimate_crr_implied_volatility,
    crr_vector_params_cont,
)
print(f"Finished Continuous in {time.time() - start} seconds")

Finished Discrete in 514.051295042038 seconds


--- Logging error ---
multiprocess.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/multiprocess/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/multiprocess/pool.py", line 48, in mapstar
    return list(map(*args))
           ^^^^^^^^^^^^^^^^
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/pathos/helpers/mp_helper.py", line 15, in <lambda>
    func = lambda args: f(*args)
                        ^^^^^^^^
  File "/Users/chiemelienwanisobi/cloned_repos/QuantTools/module_test/raw_code/optionlib_2/vol/implied_vol.py", line 158, in vector_vol_estimation
    estimated_vols = [brute_callable(*params) for params in list_input]
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ValueError: Invalid probability p=1.0109933236781294. It must be between 0 and 1.

In [43]:
aapl_chain['crr_vol'] = vol_batch_crr
aapl_chain.Expiration.sort_values().unique()

<DatetimeArray>
['2025-07-18 00:00:00', '2025-07-25 00:00:00', '2025-08-01 00:00:00',
 '2025-08-08 00:00:00', '2025-08-15 00:00:00', '2025-08-22 00:00:00',
 '2025-08-29 00:00:00', '2025-09-19 00:00:00', '2025-10-17 00:00:00',
 '2025-11-21 00:00:00', '2025-12-19 00:00:00', '2026-01-16 00:00:00',
 '2026-02-20 00:00:00', '2026-03-20 00:00:00', '2026-05-15 00:00:00',
 '2026-06-18 00:00:00', '2026-09-18 00:00:00', '2026-12-18 00:00:00',
 '2027-01-15 00:00:00', '2027-06-17 00:00:00', '2027-12-17 00:00:00']
Length: 21, dtype: datetime64[ns]

In [52]:
aapl_chain.Expiration.unique()
aapl_chain[(aapl_chain['Expiration'] == '2027-12-17') & (aapl_chain['Right'] == 'C')].sort_values('Strike').tail(50).plot(y = ['crr_vol', 
                                                                                                                               'bjs_vol'], x='Strike', kind='line', title='AAPL Call Options Implied Volatility')

In [60]:


bjs_greeks = bjs2002_numerical_greeks(
    K=aapl_chain['Strike'].tolist(),
    T=T,
    r=r,
    sigma=aapl_chain['bjs_vol'].tolist(),
    S=s,
    div_yield=q,
    option_type=aapl_chain['Right'].str.lower().tolist(),
)
bjs_greeks['delta']

array([-1.66211164,  0.44585662,  0.43537571, ..., -1.95918893,
       -1.3861209 ,  0.61119484])

In [72]:
binomial_greeks = binomial_tree_greeks(
    K=aapl_chain['Strike'].tolist(),
    expiration= aapl_chain['Expiration'].tolist(),
    r=r,
    sigma=aapl_chain['crr_vol'].tolist(),
    S=s,
    div_amount=discrete_q_convert,
    option_type=aapl_chain['Right'].str.lower().tolist(),
    N=100,  # Number of steps in the binomial tree
    dividend_type='discrete',  # Dividend type
    valuation_date=test_valuation_date,  # Valuation date
    american=True,  # American options
    start_date=test_valuation_date,  # Start date for dividends

)


Lengths: K=2428, expiration=2428, sigma=2428, r=2428, N=2428, S=2428, dividend_type=2428, option_type=2428, start_date=2428, valuation_date=2428, american=2428


In [62]:
binomial_tree_greeks?

Signature:
binomial_tree_greeks(
    K: float | numpy.ndarray,
    expiration: float | numpy.ndarray,
    sigma: float | numpy.ndarray,
    r: float | numpy.ndarray,
    N: float | numpy.ndarray,
    S: float | numpy.ndarray,
    dividend_type: float | numpy.ndarray,
    div_amount: float | numpy.ndarray,
    option_type: float | numpy.ndarray,
    start_date: float | numpy.ndarray,
    valuation_date: float | numpy.ndarray,
    american: float | numpy.ndarray,
)
Docstring:
Calculate Greeks using a binomial tree model.

Parameters:
- K: Strike price
- expiration: Expiration date of the option
- sigma: Volatility of the underlying asset
- r: Risk-free interest rate
- N: Number of time steps in the binomial tree
- spot_price: Current price of the underlying asset (optional)
- dividend_type: Type of dividend ('discrete' or 'continuous')
- div_amount: Amount of dividend (if applicable)
- option_type: 'c' for call, 'p' for put
- start_date: Start date for the option pricing (optional)
- val